In [100]:
import pandas as pd

In [101]:
riders = pd.read_csv("../data/Ridership/ridership.csv")

#don't care about origin and destination of same station, no one does this, ridership is always 0
riders = riders[riders["Origin_ID"] != riders['Dest_ID']]

#don't care about weekend ridership because we don't have schedule data for weekends
riders = riders[riders['TOD'] != 'wknd']

#make ridership column a number not a string
riders['Ridership_Number'] = riders['Ridership_Number'].str.replace(',', '')
riders = riders.astype({'Ridership_Number': 'int64'})

#reduce to only the scenarios we care about
riders = riders[((riders['Scenario'] == 'Ridership_2017_observed') | 
                  (riders['Scenario'] == 'Ridership_2040_HSR2') |
                  (riders['Scenario'] == 'Ridership_2040_moderate') |
                  (riders['Scenario'] == 'Ridership_2040_high'))]

In [104]:
trains = pd.read_csv("../data/parsed_data/schedules/combined_schedules_metrics.csv")

#filter to just the information we care about
trains = trains[['scenario', 'departure_hour',
       'count_journey_time', 'departure_station', 'arrival_station']]

#again nobody rides from origin and destination of the same station.
trains = trains[trains['arrival_station'] != trains['departure_station']]

trains=trains.drop_duplicates()

#match up scenario names and times of day with the ridership info
scenario_dict = {'Baseline': 'Ridership_2040_HSR2', 
                 'Existing': 'Ridership_2017_observed', 
                 'High': 'Ridership_2040_high', 
                 'HIgh': 'Ridership_2040_high', 
                 'Moderate': 'Ridership_2040_moderate'}

#add time of day information to the trains
time_dict = {4: 'ea', 5: 'ea', 
             6: 'am', 7: 'am', 8: 'am', 9: 'am', 10: 'am', 
             11: 'md', 12: 'md', 13: 'md', 14: 'md', 
             15: 'pm', 16: 'pm', 17: 'pm', 18: 'pm', 19: 'pm', 
             20: 'ev', 21: 'ev', 22: 'ev', 23: 'ev', 24: 'ev', 25: 'ev'}

trains['Scenario'] = trains['scenario'].apply(lambda x: scenario_dict[x])
trains = trains.drop(columns='scenario')
trains['TOD'] = trains['departure_hour'].apply(lambda x: time_dict[x])

#to get trains for each time of day, just group by TOD and then sum up the count of trains
trains_by_tod = trains.drop(columns='departure_hour').groupby(by = ['departure_station',
       'arrival_station', 'Scenario', 'TOD']).sum().reset_index()

In [105]:
riders.columns

Index(['TOD', 'Origin_ID', 'Origin_Name', 'Dest_ID', 'Dest_Name', 'Scenario',
       'Ridership_Number'],
      dtype='object')

In [106]:
trains.columns

Index(['departure_hour', 'count_journey_time', 'departure_station',
       'arrival_station', 'Scenario', 'TOD'],
      dtype='object')

In [107]:
new_trains[(new_trains['departure_station'] == 'South San Francisco') & 
       (new_trains['arrival_station'] == 'Belmont') &
       (new_trains['departure_hour'] == 11)]

,scenario,departure_station,arrival_station,departure_hour,count_journey_time,mean_journey_time,min_journey_time,max_journey_time,count_wait_time,mean_wait_time,...,max_wait_time,lat_departure_station,lon_departure_station,SB_order_departure_station,NB_order_departure_station,lat_arrival_station,lon_arrival_station,SB_order_arrival_station,NB_order_arrival_station,isochrone_circle_distance
8607,High,South San Francisco,Belmont,11,2,23.0,23,23,60,18.25,...,44.0,37.654343,-122.406324,4,27,37.521203,-122.276244,12,19,3108.0
8626,Moderate,South San Francisco,Belmont,11,1,21.0,21,21,60,29.50,...,59.0,37.654343,-122.406324,4,27,37.521203,-122.276244,12,19,3276.0
8645,Existing,South San Francisco,Belmont,11,1,27.0,27,27,60,29.50,...,59.0,37.654343,-122.406324,4,27,37.521203,-122.276244,12,19,2772.0


In [108]:
riders_trains_by_tod = pd.merge(riders, trains_by_tod,  how='inner', 
                         left_on=['TOD','Scenario','Origin_Name','Dest_Name'], 
                         right_on = ['TOD','Scenario','departure_station','arrival_station'])

riders_trains_by_tod = riders_trains_by_tod.drop(columns = ['departure_station','arrival_station'])

In [109]:
#this merge will add a station to every row (and repeat that row for each station)
df_tod = riders_trains_by_tod.copy()
df2 = riders_trains_by_tod[['TOD', 'Scenario', 'Origin_Name', 'Origin_ID']]
df2 = df2.rename(columns={"Origin_Name":"Station_Name", "Origin_ID": "Station_ID"})
df2 = df2.drop_duplicates()

df_tod = df_tod.merge(df2, on=['Scenario', 'TOD'])

In [110]:
#separate northbound and southbound trips
sb_tod = df_tod[df_tod['Origin_ID'] < df_tod['Dest_ID']]
nb_tod = df_tod[df_tod['Origin_ID'] > df_tod['Dest_ID']]

#now we only care about the stations that are equal to or between the origin and destination - 
#those riders are people who are on the train at that station
sb_tod = sb_tod[(sb_tod['Station_ID'] >= sb_tod['Origin_ID']) & (sb_tod['Station_ID'] <= sb_tod['Dest_ID'])]
nb_tod = nb_tod[(nb_tod['Station_ID'] <= nb_tod['Origin_ID']) & (nb_tod['Station_ID'] >= nb_tod['Dest_ID'])]

#group by station and sum up how many riders on the train at that station
sb_tod_grouped = sb_tod.groupby(['TOD', 'Scenario', 'Station_Name']).agg({'Ridership_Number': 'sum', 'count_journey_time': 'max'}).reset_index()
nb_tod_grouped = nb_tod.groupby(['TOD', 'Scenario', 'Station_Name']).agg({'Ridership_Number': 'sum', 'count_journey_time': 'max'}).reset_index()

#calculate number of riders per train
sb_tod_grouped['per_train'] = sb_tod_grouped['Ridership_Number']/sb_tod_grouped['count_journey_time']
nb_tod_grouped['per_train'] = nb_tod_grouped['Ridership_Number']/nb_tod_grouped['count_journey_time']

In [47]:
sb_tod_grouped.sample(20)

,TOD,Scenario,Station_Name,Ridership_Number,count_journey_time,per_train
479,pm,Ridership_2040_HSR2,Menlo Park,24090,50,481.800000
295,ev,Ridership_2040_high,San Mateo,5290,32,165.312500
501,pm,Ridership_2040_high,Broadway,20283,40,507.075000
393,md,Ridership_2040_high,Menlo Park,2741,28,97.892857
147,ea,Ridership_2040_HSR2,Menlo Park,154,10,15.400000
472,pm,Ridership_2040_HSR2,Burlingame,26925,50,538.500000
548,pm,Ridership_2040_moderate,San Jose Diridon,10430,44,237.045455
15,am,Ridership_2017_observed,San Carlos,4152,15,276.800000
90,am,Ridership_2040_moderate,California Ave,6635,40,165.875000
304,ev,Ridership_2040_moderate,Bayshore,5985,31,193.064516


In [111]:
'''
Ridership is given by Time of day (TOD) blocks. Train schedule is more precise than this.
The below maps can be used to divide up ridership into hourly blocks for better visualizations.

Hour to TOD map:
{4: 'ea', 5: 'ea', 
    6: 'am', 7: 'am', 8: 'am', 9: 'am', 10: 'am', 
    11: 'md', 12: 'md', 13: 'md', 14: 'md', 
    15: 'pm', 16: 'pm', 17: 'pm', 18: 'pm', 19: 'pm', 
    20: 'ev', 21: 'ev', 22: 'ev', 23: 'ev', 24: 'ev', 25: 'ev'}
'''

# a uniform distribution of riders within each time block
hourly_division_uniform = {4: 0.5, 5: 0.5, 
             6: 0.2, 7: 0.2, 8: 0.2, 9: 0.2, 10: 0.2, 
             11: 0.25, 12: 0.25, 13: 0.25, 14: 0.25, 
             15: 0.2, 16: 0.2, 17: 0.2, 18: 0.2, 19: 0.2, 
             20: 0.2, 21: 0.2, 22: 0.2, 23: 0.2, 24: 0.2, 25: 0.0}

# a smoothed distribution of riders that creates peaks for the morning/evening commutes 
# and attempts to even out jumps between TOD blocks
hourly_division_smoothed = {4: 0.5, 5: 0.5, 
             6: 0.1, 7: 0.25, 8: 0.3, 9: 0.25, 10: 0.1, 
             11: 0.25, 12: 0.25, 13: 0.25, 14: 0.25, 
             15: 0.1, 16: 0.25, 17: 0.3, 18: 0.25, 19: 0.1, 
             20: 0.3, 21: 0.25, 22: 0.2, 23: 0.1, 24: 0.1, 25: 0.05}

In [112]:
riders_trains_by_hour = pd.merge(riders, trains,  how='inner', 
                         left_on=['TOD','Scenario','Origin_Name','Dest_Name'], 
                         right_on = ['TOD','Scenario','departure_station','arrival_station'])

riders_trains_by_hour = riders_trains_by_hour.drop(columns = ['departure_station','arrival_station'])

In [113]:
#this merge will add a station to every row (and repeat that row for each station)
df_hourly = riders_trains_by_hour.copy()
df2 = riders_trains_by_hour[['TOD', 'Scenario', 'Origin_Name', 'Origin_ID']]
df2 = df2.rename(columns={"Origin_Name":"Station_Name", "Origin_ID": "Station_ID"})
df2 = df2.drop_duplicates()

df_hourly = df_hourly.merge(df2, on=['Scenario', 'TOD'])

In [114]:
#separate northbound and southbound trips
sb_hourly = df_hourly[df_hourly['Origin_ID'] < df_hourly['Dest_ID']]
nb_hourly = df_hourly[df_hourly['Origin_ID'] > df_hourly['Dest_ID']]

#now we only care about the stations that are equal to or between the origin and destination - 
#those riders are people who are on the train at that station

sb_hourly = sb_hourly[(sb_hourly['Station_ID'] >= sb_hourly['Origin_ID']) & 
                      (sb_hourly['Station_ID'] <= sb_hourly['Dest_ID'])]
nb_hourly = nb_hourly[(nb_hourly['Station_ID'] <= nb_hourly['Origin_ID']) & 
                      (nb_hourly['Station_ID'] >= nb_hourly['Dest_ID'])]

#group by station and sum up how many riders on the train at that station
sb_hourly_grouped = sb_hourly.groupby(['departure_hour', 'Scenario', 'Station_Name']).agg({'Ridership_Number': 'sum', 'count_journey_time': 'max'}).reset_index()
nb_hourly_grouped = nb_hourly.groupby(['departure_hour', 'Scenario', 'Station_Name']).agg({'Ridership_Number': 'sum', 'count_journey_time': 'max'}).reset_index()



In [115]:
#for the uniform distribution
#divide up ridership by hour
sb_hourly_grouped['Hourly_Ridership_uniform'] = sb_hourly_grouped['departure_hour'].apply(lambda x: hourly_division_uniform[x])*sb_hourly_grouped['Ridership_Number']
nb_hourly_grouped['Hourly_Ridership_uniform'] = nb_hourly_grouped['departure_hour'].apply(lambda x: hourly_division_uniform[x])*nb_hourly_grouped['Ridership_Number']

#calculate number of riders per train
sb_hourly_grouped['per_train_uniform'] = sb_hourly_grouped['Hourly_Ridership_uniform']/sb_hourly_grouped['count_journey_time']
nb_hourly_grouped['per_train_uniform'] = nb_hourly_grouped['Hourly_Ridership_uniform']/nb_hourly_grouped['count_journey_time']


In [116]:
#for the smoothed distribution
#divide up ridership by hour
sb_hourly_grouped['Hourly_Ridership_smoothed'] = sb_hourly_grouped['departure_hour'].apply(lambda x: hourly_division_smoothed[x])*sb_hourly_grouped['Ridership_Number']
nb_hourly_grouped['Hourly_Ridership_smoothed'] = nb_hourly_grouped['departure_hour'].apply(lambda x: hourly_division_smoothed[x])*nb_hourly_grouped['Ridership_Number']

#calculate number of riders per train
sb_hourly_grouped['per_train_smoothed'] = sb_hourly_grouped['Hourly_Ridership_smoothed']/sb_hourly_grouped['count_journey_time']
nb_hourly_grouped['per_train_smoothed'] = nb_hourly_grouped['Hourly_Ridership_smoothed']/nb_hourly_grouped['count_journey_time']

In [117]:
sb_hourly_grouped.sample(20)

,departure_hour,Scenario,Station_Name,Ridership_Number,count_journey_time,Hourly_Ridership_uniform,per_train_uniform,Hourly_Ridership_smoothed,per_train_smoothed
1000,13,Ridership_2040_HSR2,Sunnyvale,1644,5,411.00,82.200000,411.00,82.200000
2162,23,Ridership_2040_high,Lawrence,2386,9,477.20,53.022222,238.60,26.511111
1237,15,Ridership_2040_high,Broadway,32268,12,6453.60,537.800000,3226.80,268.900000
1347,16,Ridership_2040_HSR2,Tamien,3392,6,678.40,113.066667,848.00,141.333333
1728,19,Ridership_2040_high,Santa Clara,14767,12,2953.40,246.116667,1476.70,123.058333
1398,16,Ridership_2040_moderate,Redwood City,29439,12,5887.80,490.650000,7359.75,613.312500
682,10,Ridership_2040_high,Burlingame,16873,11,3374.60,306.781818,1687.30,153.390909
329,7,Ridership_2040_HSR2,South San Francisco,13390,10,2678.00,267.800000,3347.50,334.750000
1492,17,Ridership_2040_high,Santa Clara,14767,12,2953.40,246.116667,4430.10,369.175000
1214,15,Ridership_2040_HSR2,Lawrence,14649,10,2929.80,292.980000,1464.90,146.490000


In [118]:
locs = pd.read_csv("../data/station_lat_lon_key.csv")

In [119]:
def add_location_data(ridership_df, locations_df):
    
    #drop unneeded data from locations, reset index for join
    locs = locations_df.drop(columns=["nasty_station", "SB_order", "NB_order"]).set_index("pretty_station")
    
    #join location and ridership on station name (station of ridership origin)
    data = ridership_df.join(locs, on="Station_Name")
    
#     #join location and ridership on station name again, this time for ridership destination
#     #so there are now two sets of location data in each row, for both origin and destination
#     data = data.join(locs, on="Dest_Name", rsuffix="_dest")
    
#     #rename some columns for clarity
#     data = data.rename(columns={"lat":"lat_origin", 
#                              "lon":"lon_origin", 
#                              "SB_order":"SB_order_origin", 
#                              "NB_order":"NB_order_origin"
#                                })
    
    return data

In [120]:
sb_tod_data = add_location_data(sb_tod_grouped, locs)
nb_tod_data = add_location_data(nb_tod_grouped, locs)
sb_hourly_data = add_location_data(nb_hourly_grouped, locs)
nb_hourly_data = add_location_data(nb_hourly_grouped, locs)

In [123]:
sb_hourly_data.columns

Index(['departure_hour', 'Scenario', 'Station_Name', 'Ridership_Number',
       'count_journey_time', 'Hourly_Ridership_uniform', 'per_train_uniform',
       'Hourly_Ridership_smoothed', 'per_train_smoothed', 'lat', 'lon'],
      dtype='object')

In [130]:
len(sb_hourly_data)

2270

In [131]:
len(nb_hourly_data)

2270

In [121]:
sb_tod_data.to_csv("../data/Ridership/sb_ridership_bytod_bystation.csv", index=False)
nb_tod_data.to_csv("../data/Ridership/nb_ridership_bytod_bystation.csv", index=False)
sb_hourly_data.to_csv("../data/Ridership/sb_ridership_byhour_bystation.csv", index=False)
nb_hourly_data.to_csv("../data/Ridership/nb_ridership_byhour_bystation.csv", index=False)

In [141]:
total_hourly = pd.merge(sb_hourly_data, nb_hourly_data, on = ['departure_hour', 
                                                              'Scenario', 'Station_Name', 
                                                              'lat', 'lon']
                       )#.sum().reset_index()#.drop(columns = ['per_train_uniform', 'per_train_smoothed'])


In [142]:
total_hourly.head(20)

,departure_hour,Scenario,Station_Name,Ridership_Number_x,count_journey_time_x,Hourly_Ridership_uniform_x,per_train_uniform_x,Hourly_Ridership_smoothed_x,per_train_smoothed_x,lat,lon,Ridership_Number_y,count_journey_time_y,Hourly_Ridership_uniform_y,per_train_uniform_y,Hourly_Ridership_smoothed_y,per_train_smoothed_y
0,4,Ridership_2017_observed,22nd St,73,1,36.5,36.5,36.5,36.5,37.757576,-122.392412,73,1,36.5,36.5,36.5,36.5
1,4,Ridership_2017_observed,4th and King,65,1,32.5,32.5,32.5,32.5,37.776738,-122.394947,65,1,32.5,32.5,32.5,32.5
2,4,Ridership_2017_observed,Bayshore,73,1,36.5,36.5,36.5,36.5,37.707716,-122.401817,73,1,36.5,36.5,36.5,36.5
3,4,Ridership_2017_observed,Belmont,100,1,50.0,50.0,50.0,50.0,37.521203,-122.276244,100,1,50.0,50.0,50.0,50.0
4,4,Ridership_2017_observed,Burlingame,98,1,49.0,49.0,49.0,49.0,37.579784,-122.344069,98,1,49.0,49.0,49.0,49.0
5,4,Ridership_2017_observed,California Ave,140,1,70.0,70.0,70.0,70.0,37.428925,-122.141309,140,1,70.0,70.0,70.0,70.0
6,4,Ridership_2017_observed,Hayward Park,98,1,49.0,49.0,49.0,49.0,37.552945,-122.309426,98,1,49.0,49.0,49.0,49.0
7,4,Ridership_2017_observed,Hillsdale,98,1,49.0,49.0,49.0,49.0,37.541572,-122.300689,98,1,49.0,49.0,49.0,49.0
8,4,Ridership_2017_observed,Lawrence,79,1,39.5,39.5,39.5,39.5,37.370542,-121.996786,79,1,39.5,39.5,39.5,39.5
9,4,Ridership_2017_observed,Menlo Park,103,1,51.5,51.5,51.5,51.5,37.454817,-122.182345,103,1,51.5,51.5,51.5,51.5


In [143]:
len(total_hourly)

2270

In [144]:
sb_hourly_data.columns

Index(['departure_hour', 'Scenario', 'Station_Name', 'Ridership_Number',
       'count_journey_time', 'Hourly_Ridership_uniform', 'per_train_uniform',
       'Hourly_Ridership_smoothed', 'per_train_smoothed', 'lat', 'lon'],
      dtype='object')

In [149]:
sb_hourly_clean = sb_hourly_data[['departure_hour', 'Scenario', 'Station_Name',
       'Hourly_Ridership_smoothed', 'per_train_smoothed', 'lat', 'lon']].rename(columns={'Hourly_Ridership_smoothed':'Hourly_Ridership', 
                                                                                         'per_train_smoothed': 'Riders_per_train'})

nb_hourly_clean = nb_hourly_data[['departure_hour', 'Scenario', 'Station_Name',
       'Hourly_Ridership_smoothed', 'per_train_smoothed', 'lat', 'lon']].rename(columns={'Hourly_Ridership_smoothed':'Hourly_Ridership', 
                                                                                         'per_train_smoothed': 'Riders_per_train'})

In [150]:
sb_hourly_clean.to_csv("../data/Ridership/sb_ridership_byhour_bystation_clean.csv", index=False)
nb_hourly_clean.to_csv("../data/Ridership/nb_ridership_byhour_bystation_clean.csv", index=False)